# Séance 2 - Bonus : ResNet à la main

Pour poursuivre le travail du TP, on se propose d'explorer une autre manière de définir un réseau de neurones au travers d'une architecture classique de Deep Learning pour la vision : les ResNet. 

Commençons par importer et traiter les données.

In [16]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

(X_train_full, y_train_full), (X_test, y_test) = (keras.datasets.fashion_mnist.load_data())
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, train_size=0.8)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.astype(np.float32).reshape(-1, 28 * 28)).reshape(-1, 28, 28, 1)
X_valid = scaler.transform(X_valid.astype(np.float32).reshape(-1, 28 * 28)).reshape(-1, 28, 28, 1)
X_test = scaler.transform(X_test.astype(np.float32).reshape(-1, 28 * 28)).reshape(-1, 28, 28, 1)

Pour rappel, une architecture ResNet correspond à la succession de ResBlock qui, dans le papier d'origine, ont l'architecture suivante :
1. Une couche de convolution avec padding pour conserver la taille de l'image
2. Une couche de BatchNormalization, que l'on explicitera en cours à la séance prochaine
3. L'activation ReLU, qui ne doit donc pas être présente dans la couche de convolution
4. Une couche de convolution avec padding pour conserver la taille de l'image
5. Une couche de BatchNormalization
6. Un ajout de l'input avant le point 1 et du résultat de la dernière couche de BatchNormalization (point 5)
7. L'activation ReLU sur l'ajout

On ne peut pas définir cette architecture si l'on utilise la méthode d'instanciation que l'on a utilisé jusqu'ici. Pour y arriver, nous avons besoin d'utiliser la version *fonctionnelle* de Keras. Par exemple avec un modèle dense :

In [17]:
input = keras.layers.Input(shape=X_train.shape[1:])
flatten = keras.layers.Flatten()(input)
dense_1 = keras.layers.Dense(units=64, activation="relu")(flatten)
dense_2 = keras.layers.Dense(units=64)(dense_1)
activation = keras.layers.ReLU()(dense_2)
output = keras.layers.Dense(units=10, activation="softmax")(activation)

model = keras.models.Model(inputs=[input], outputs=[output])
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,050 (215.04 KB)

 Trainable params: 55,050 (215.04 KB)

 Non-trainable params: 0 (0.00 B)

**Consigne** : En utilisant la version fonctionnelle de Keras, définir une fonction `ResidualBlock` qui prend en paramètre un argument *input* qui correspondra à une couche Keras et en second argument des *kwargs* à passer aux différentes couches de convolutions.

In [18]:
def ResidualBlock(input, **kwargs):
    convolution = keras.layers.Conv2D(**kwargs)(input)
    normalization = keras.layers.BatchNormalization()(convolution)
    activation = keras.layers.ReLU()(normalization)

    convolution = keras.layers.Conv2D(**kwargs)(activation)
    normalization = keras.layers.BatchNormalization()(convolution)

    addition = keras.layers.Add()([input, normalization])
    output = keras.layers.ReLU()(addition)
    return output

**Consigne** : Définir un modèle utilisant deux ResBlock pour résoudre le problème de classification auquel on s'intéresse.

In [19]:
input = keras.layers.Input(shape=X_train.shape[1:])
hidden = keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding="same", kernel_initializer="he_normal")(input)
starter_output = keras.layers.MaxPooling2D(pool_size=2)(hidden)

block = ResidualBlock(input=starter_output, filters=64, kernel_size=3, padding="same", kernel_initializer="he_normal")
block = ResidualBlock(input=block, filters=64, kernel_size=3, padding="same", kernel_initializer="he_normal")

pooling = keras.layers.MaxPooling2D(pool_size=2)(block)
flatten = keras.layers.Flatten()(pooling)
hidden = keras.layers.Dense(256, activation="relu", kernel_initializer="he_normal")(flatten)
output = keras.layers.Dense(10, activation="softmax", kernel_initializer="he_normal")(hidden)

model = keras.models.Model(inputs=[input], outputs=[output])

**Consigne** : Après avoir compilé le modèle, lancer l'entraînement sur quelque époque pour vérifier qu'il fonctionne.

In [20]:
n_epochs = 5
learning_rate = 1e-3
batch_size = 256

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=learning_rate), metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_data=(X_valid, y_valid))

Epoch 1/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 16s 66ms/step - accuracy: 0.5891 - loss: 1.3718 - val_accuracy: 0.7704 - val_loss: 0.6414
Epoch 2/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 12s 64ms/step - accuracy: 0.7847 - loss: 0.6072 - val_accuracy: 0.8113 - val_loss: 0.5368
Epoch 3/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.8177 - loss: 0.5168 - val_accuracy: 0.8318 - val_loss: 0.4851
Epoch 4/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 12s 64ms/step - accuracy: 0.8301 - loss: 0.4790 - val_accuracy: 0.8415 - val_loss: 0.4571
Epoch 5/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.8429 - loss: 0.4446 - val_accuracy: 0.8455 - val_loss: 0.4358


Pour aller plus loin, on pourrait se demander s'il est nécessaire de réaliser ces connexions résiduelles. Pour y répondre, on conseille de construire un réseau de neurones convolutionnel *classique* et comparer les performances pour plusieurs entraînement.